In [2]:
import cv2
import torch
import numpy as np
import time


# Cargar modelo YOLOv5 preentrenado una sola vez
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.to('cpu')  # Forzar uso de CPU
model.eval()  # Poner el modelo en modo evaluación para mejorar rendimiento

Using cache found in /home/silvia/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-2-8 Python-3.12.4 torch-2.6.0+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [3]:
# Lista de objetos clave para el escape room
OBJETOS_CLAVE = {
    "key": "UNA LLAVE!",
    "book": "UN LIBRO", 
    "pen": "UN BOLIGRAFO",
    "pencil": "UN LAPIZ",
    "paper":
    "notebook",
    "cup": "UNA TAZA!",
    "cell phone": "UN TELEFONO MOVIL!"
}

In [4]:
# Diccionario para almacenar el conteo de cada objeto detectado
deteccion_continua = {obj: 0 for obj in OBJETOS_CLAVE.keys()}
UMBRAL_DETECCION = 5  # Frames consecutivos necesarios para confirmar detección
TIEMPO_ESPERA = 2  # Segundos de espera después de detectar un objeto estable

In [12]:
def detect_objects():
    cap = cv2.VideoCapture(0)  # Iniciar webcam
    if not cap.isOpened():
        print("Error: No se pudo acceder a la webcam")
        return
    
    ventana_abierta = False  # Para controlar si la ventana está abierta
    imagen_capturada = None  # Para almacenar la imagen capturada

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        
        # Reducir la resolución para mejorar rendimiento
        frame = cv2.resize(frame, (320, 240))

        # Convertir la imagen a RGB (YOLO usa imágenes en RGB)
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Realizar la detección de objetos
        results = model(img_rgb)
        detections = results.pandas().xyxy[0]  # Obtener los resultados en DataFrame

        # Resetear detecciones en cada frame
        objetos_detectados = {obj: False for obj in OBJETOS_CLAVE.keys()}
        texto_detectado = None  # Para almacenar el texto a mostrar

        for _, row in detections.iterrows():
            obj_name = row['name']
            confidence = row['confidence']

            if obj_name in OBJETOS_CLAVE and confidence > 0.5:
                objetos_detectados[obj_name] = True
                x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
                label = f"{OBJETOS_CLAVE[obj_name]} {confidence:.2f}"

                # Dibujar los rectángulos y etiquetas en la imagen
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Manejo de detecciones continuas
        for obj in OBJETOS_CLAVE.keys():
            if objetos_detectados[obj]:
                deteccion_continua[obj] += 1
            else:
                deteccion_continua[obj] = max(0, deteccion_continua[obj] - 1)  # Evita valores negativos

            # Si el objeto ha sido detectado en varios frames seguidos, activar evento
            if deteccion_continua[obj] >= UMBRAL_DETECCION:
                texto_detectado = OBJETOS_CLAVE[obj]  # Guardar el texto del objeto detectado
                print(f"¡Se ha detectado {texto_detectado} de forma estable!")

                # Capturar la imagen en ese momento
                imagen_capturada = frame.copy()

                # Dibujar el mensaje en la pantalla
                cv2.putText(frame, texto_detectado, (20, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

                # Mostrar la imagen con el texto antes de cerrar la webcam
                #cv2.imshow('Detección Confirmada', imagen_capturada)
                #cv2.waitKey(2000)  # Mostrar por 2 segundos

                cap.release()  # Cerrar la webcam
                cv2.destroyAllWindows()
                ventana_abierta = False  # Resetear el estado de la ventana

                # Mostrar la imagen capturada
                if imagen_capturada is not None:
                    cv2.imshow('Imagen Capturada', imagen_capturada)
                    if  cv2.waitKey(0): # Esperar a que el usuario cierre la imagen
                        break
            #return  # Salir de la función después de mostrar la imagen capturada

        # Abrir la ventana solo si no está abierta
        if not ventana_abierta:
            cv2.namedWindow('Detección de Objetos - Escape Room', cv2.WINDOW_NORMAL)
            ventana_abierta = True

        # Mostrar la imagen con los objetos detectados
        cv2.imshow('Detección de Objetos - Escape Room', frame)

        #Salir con la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    ventana_abierta = False  # Resetear el estado de la ventana


In [13]:
detect_objects()

/home/silvia/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
QObject::moveToThread: Current thread (0x108d9e00) is not the object's thread (0x10af5340).
Cannot move to target thread (0x108d9e00)

QObject::moveToThread: Current thread (0x108d9e00) is not the object's thread (0x10af5340).
Cannot move to target thread (0x108d9e00)

QObject::moveToThread: Current thread (0x108d9e00) is not the object's thread (0x10af5340).
Cannot move to target thread (0x108d9e00)

QObject::moveToThread: Current thread (0x108d9e00) is not the object's thread (0x10af5340).
Cannot move to target thread (0x108d9e00)

QObject::moveToThread: Current thread (0x108d9e00) is not the object's thread (0x10af5340).
Cannot move to target thread (0x108d9e00)

QObject::moveToThread: Current thread (0x108d9e00) is not the object's thread (0x10af5340).
C

¡Se ha detectado UN TELEFONO MOVIL! de forma estable!


QObject::moveToThread: Current thread (0x108d9e00) is not the object's thread (0x10af5340).
Cannot move to target thread (0x108d9e00)

QObject::moveToThread: Current thread (0x108d9e00) is not the object's thread (0x10af5340).
Cannot move to target thread (0x108d9e00)

QObject::moveToThread: Current thread (0x108d9e00) is not the object's thread (0x10af5340).
Cannot move to target thread (0x108d9e00)

QObject::moveToThread: Current thread (0x108d9e00) is not the object's thread (0x10af5340).
Cannot move to target thread (0x108d9e00)

QObject::moveToThread: Current thread (0x108d9e00) is not the object's thread (0x10af5340).
Cannot move to target thread (0x108d9e00)

QObject::moveToThread: Current thread (0x108d9e00) is not the object's thread (0x10af5340).
Cannot move to target thread (0x108d9e00)

QObject::moveToThread: Current thread (0x108d9e00) is not the object's thread (0x10af5340).
Cannot move to target thread (0x108d9e00)

QObject::moveToThread: Current thread (0x108d9e00) is n

In [13]:
# para saber qué etiquetas me detecta YOLO

import torch

# Cargar modelo YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Obtener la lista de etiquetas de objetos que puede detectar
clases_yolo = model.names

# Imprimir la lista completa de clases
print("Lista de objetos detectables por YOLOv5:")
for i, nombre in enumerate(clases_yolo):
    print(f"{i}: {nombre}")


print(model.names)

Using cache found in /home/silvia/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-2-8 Python-3.12.4 torch-2.6.0+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Lista de objetos detectables por YOLOv5:
0: 0
1: 1
2: 2
3: 3
4: 4
5: 5
6: 6
7: 7
8: 8
9: 9
10: 10
11: 11
12: 12
13: 13
14: 14
15: 15
16: 16
17: 17
18: 18
19: 19
20: 20
21: 21
22: 22
23: 23
24: 24
25: 25
26: 26
27: 27
28: 28
29: 29
30: 30
31: 31
32: 32
33: 33
34: 34
35: 35
36: 36
37: 37
38: 38
39: 39
40: 40
41: 41
42: 42
43: 43
44: 44
45: 45
46: 46
47: 47
48: 48
49: 49
50: 50
51: 51
52: 52
53: 53
54: 54
55: 55
56: 56
57: 57
58: 58
59: 59
60: 60
61: 61
62: 62
63: 63
64: 64
65: 65
66: 66
67: 67
68: 68
69: 69
70: 70
71: 71
72: 72
73: 73
74: 74
75: 75
76: 76
77: 77
78: 78
79: 79
{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee

In [7]:
print(model.names)

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [17]:
# Convertir a DataFrame para mostrar en formato de columna
import pandas as pd
df_clases = pd.DataFrame({"ID": list(clases_yolo.keys()), "Objeto": list(clases_yolo.values())})

# Mostrar en formato columna
print(df_clases)

    ID          Objeto
0    0          person
1    1         bicycle
2    2             car
3    3      motorcycle
4    4        airplane
5    5             bus
6    6           train
7    7           truck
8    8            boat
9    9   traffic light
10  10    fire hydrant
11  11       stop sign
12  12   parking meter
13  13           bench
14  14            bird
15  15             cat
16  16             dog
17  17           horse
18  18           sheep
19  19             cow
20  20        elephant
21  21            bear
22  22           zebra
23  23         giraffe
24  24        backpack
25  25        umbrella
26  26         handbag
27  27             tie
28  28        suitcase
29  29         frisbee
30  30            skis
31  31       snowboard
32  32     sports ball
33  33            kite
34  34    baseball bat
35  35  baseball glove
36  36      skateboard
37  37       surfboard
38  38   tennis racket
39  39          bottle
40  40      wine glass
41  41             cup
42  42     

In [16]:
pd.set_option('display.max_rows', None)

In [19]:
import torch

# Cargar modelo YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Convertir los nombres de objetos en una lista
objetos_yolo = list(model.names.values())

# Objetos que quieres comprobar
objetos_usuario = ["key", "cup", "cell phone", "book", "pen", "pencil", "paper", "notebook", "headphones", "CDs", "watch", "PC mouse", "calculator", "plant", "sneakers", "paddle racket", "tennis racket", "cup", "fork", "knife", "spoon", "plate"]

# Comprobar cuáles de los objetos del usuario están en la lista de YOLO
objetos_encontrados = [obj for obj in objetos_usuario if obj in objetos_yolo]

# Mostrar los resultados
print("Objetos detectables en YOLO:", objetos_yolo)
print("\nObjetos del usuario:", objetos_usuario)
print("\nObjetos que sí están en YOLO:", objetos_encontrados)

# Si algún objeto no está en la lista
objetos_no_encontrados = [obj for obj in objetos_usuario if obj not in objetos_yolo]
if objetos_no_encontrados:
    print("\nObjetos que NO están en YOLO:", objetos_no_encontrados)


Using cache found in /home/silvia/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-2-8 Python-3.12.4 torch-2.6.0+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Objetos detectables en YOLO: ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

Objetos del usuario: ['key', 'cup', 'cell phone', 'book', 'pen', 'pencil', 'paper', 'notebook', 'headphones'

In [20]:
import openimages
print(dir(openimages))


['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']
